In [82]:
import numpy as np
import pandas as pd
import mysql.connector
from decimal import *
from pandas.plotting import scatter_matrix
import seaborn as sns
from ast import literal_eval
import folium
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import Ridge

In [83]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="1234"
)

print(mydb)
mycursor = mydb.cursor()
mycursor.execute("use csgo_data")
mycursor.execute("TRUNCATE TABLE player_stats")


In [84]:
df = pd.read_csv("../data-collecter-js/newOut.csv")
df.head(5)

,name,country,url,teams,maps_played,rounds_played,kd_difference,kd_ratio,rating,total_kills,image_url
0,torzsi,Hungary,https://www.hltv.org/stats/players/18072/torzsi,"MOUZ,Budapest Five,MOUZ NXT,Salamander",463,12383,1963,1.26,1.17,9629,https://img-cdn.hltv.org/playerbodyshot/vXrIz7...
1,yuurih,Brazil,https://www.hltv.org/stats/players/12553/yuurih,FURIA,1013,26421,3146,1.19,1.12,19975,https://img-cdn.hltv.org/playerbodyshot/_I8K7J...
2,vsm,Brazil,https://www.hltv.org/stats/players/16816/vsm,"DETONA,00NATION",533,14130,1485,1.16,1.12,10883,https://img-cdn.hltv.org/playerbodyshot/ZQ7XFT...
3,syrsoN,Germany,https://www.hltv.org/stats/players/7266/syrson,"BIG,BIG,ALTERNATE aTTaX,Sprout",1735,46095,4647,1.16,1.09,33692,https://img-cdn.hltv.org/playerbodyshot/G3MPAU...
4,ZywOo,France,https://www.hltv.org/stats/players/11893/zywoo,"Vitality,aAa",1011,26577,6115,1.37,1.27,22439,https://img-cdn.hltv.org/playerbodyshot/OfWl-Y...


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           100 non-null    object 
 1   country        100 non-null    object 
 2   url            100 non-null    object 
 3   teams          100 non-null    object 
 4   maps_played    100 non-null    int64  
 5   rounds_played  100 non-null    int64  
 6   kd_difference  100 non-null    int64  
 7   kd_ratio       100 non-null    float64
 8   rating         100 non-null    float64
 9   total_kills    100 non-null    int64  
 10  image_url      95 non-null     object 
dtypes: float64(2), int64(4), object(5)
memory usage: 8.7+ KB


In [86]:
df.describe()

,maps_played,rounds_played,kd_difference,kd_ratio,rating,total_kills
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,948.120000,24883.320000,2996.360000,1.192700,1.121000,18745.750000
std,402.187033,10575.801029,1533.111305,0.062569,0.036195,8030.567261
min,390.000000,10339.000000,718.000000,1.090000,1.090000,7931.000000
25%,618.750000,15995.500000,1756.750000,1.150000,1.100000,12123.250000
50%,889.500000,23424.500000,2605.500000,1.180000,1.110000,17515.000000
75%,1179.750000,31374.500000,3853.750000,1.200000,1.130000,22978.000000
max,2089.000000,54327.000000,9004.000000,1.450000,1.270000,40979.000000


In [87]:
df_numeric = df.select_dtypes(include=[np.number])
df_categoric = df.select_dtypes(exclude=[np.number])

df_numeric_cols = list(df_numeric.columns)
df_categoric_cols = list(df_categoric.columns)

In [88]:
def maxAndMinStats(dataframe, cols):
    """Extracting out players with interesting statistics"""
    def maxAndMin(dataf, column):
        return dataf.loc[dataf[column].idxmax()], dataf.loc[dataf[column].idxmin()]

    maxMinList = []

    for col in cols:
        print('Column: ', col)
        res = maxAndMin(dataframe, col)
        max_, min_ = res
        print('Max in ', col, ': ', max_['name'])
        # print('Min in ', col, ': ', min_['name'], '\n')
        # print(max_['image_url'])
        # print(max_[col])
        sql= "insert into player_stats(title, count, name, profile_url) VALUES(%s,%s,%s,%s)"
        values=(col,str(max_[col]),max_['name'],max_['image_url'])
        mycursor.execute(sql,values)
        mydb.commit()
        print(mycursor.rowcount, "record(s) inserted.")
        maxMinList.append(res)

    return maxMinList

In [89]:
maxMinStats = maxAndMinStats(df, df_numeric_cols)

Column:  maps_played
Max in  maps_played :  f0rest
1 record(s) inserted.
Column:  rounds_played
Max in  rounds_played :  f0rest
1 record(s) inserted.
Column:  kd_difference
Max in  kd_difference :  s1mple
1 record(s) inserted.
Column:  kd_ratio
Max in  kd_ratio :  sh1ro
1 record(s) inserted.
Column:  rating
Max in  rating :  ZywOo
1 record(s) inserted.
Column:  total_kills
Max in  total_kills :  f0rest
1 record(s) inserted.
